In [2]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
#from keras.layers.embeddings import Embedding
from keras.layers import *
from keras.models import Sequential
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
#from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError


In [3]:
data = pd.read_csv('./dataset.csv')

data['task_name'] = data['task_name'].str.lower()
data['subtask_name'] = data['subtask_name'].str.lower()

In [4]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [5]:
# def remove_stopwords(data):
#   # data['task_name']=data['task_name'].fillna("")
#   # data['subtask_name']=data['subtask_name'].fillna("")
#   data['task_name without stopwords'] = data['task_name'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
#   data['subtask_name without stopwords'] = data['subtask_name'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
#   return data

# def remove_tags(string):
#     result = re.sub('<.*?>','',string)
#     return result


In [6]:
# data_without_stopwords = remove_stopwords(data)
# data_without_stopwords['task_name']= data_without_stopwords['task_name without stopwords'].apply(lambda cw : remove_tags(cw))
# data_without_stopwords['task_name'] = data_without_stopwords['task_name'].str.replace('[{}]'.format(string.punctuation), ' ')

# data_without_stopwords['subtask_name']= data_without_stopwords['subtask_name without stopwords'].apply(lambda cw : remove_tags(cw))
# data_without_stopwords['subtask_name'] = data_without_stopwords['subtask_name'].str.replace('[{}]'.format(string.punctuation), ' ')
# data_without_stopwords.head()

In [7]:
reviews = data['subtask_name']

# integer encode
print(data.head())
label_encoder_task = LabelEncoder()
label_encoder_subtask = LabelEncoder()
data['task_name'] = label_encoder_task.fit_transform(data['task_name'])
data['subtask_name'] = label_encoder_subtask.fit_transform(data['subtask_name'])
# print(f"classes = {label_encoder_task.classes_}")
# print(f"classes = {label_encoder_subtask.classes_}")
print(data.head())
# print(data)
import pickle
pickle.dump(label_encoder_task, open("../bzkRestApis/scaler_task.sav", "wb"))
pickle.dump(label_encoder_subtask, open("../bzkRestApis/scaler_subtask.sav", "wb"))
# print(f"classes = {label_encoder_task.classes_}")
# print(f"classes = {label_encoder_subtask.classes_}")

# data['task_name'] = label_encoder_task.transform(data['task_name'])
# data['subtask_name'] = label_encoder_subtask.transform(data['subtask_name'])
# print(data.head())

# reviews
reviews_list = []
for i in range(len(reviews)):
    reviews_list.append(reviews[i])

   Skill  Complexity        task_name  \
0      3           1        home page   
1      2           0        home page   
2      3           1  categories page   
3      2           0  categories page   
4      2           0  categories page   

                                        subtask_name  Duration  
0             api to validate and fetch hot products         3  
1             api to validate and fetch new products         2  
2  api to validate and fetch all categories along...         3  
3        api to validate and fetch a single category         2  
4                     api to filter and sort results         2  
   Skill  Complexity  task_name  subtask_name  Duration
0      3           1          5            14         3
1      2           0          5            16         2
2      3           1          2             8         3
3      2           0          2             7         2
4      2           0          2             2         2


In [8]:
duration = data['Duration']
y = np.array(list(duration))
# y

In [9]:
import pickle
X_train, X_test, Y_train, Y_test = train_test_split(data[["Skill", "Complexity", "task_name", "subtask_name"]], y, test_size=0.2, random_state = 45)
# X_test


In [10]:
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train)

In [11]:
# words_to_index = tokenizer.word_index
# for key, value in words_to_index.items():
#     words_to_index[key] = words_to_index[key] - 1
# words_to_index

In [12]:
# def read_glove_vector(glove_vec):
#   with open(glove_vec, 'r', encoding='UTF-8') as f:
#     words = set()
#     word_to_vec_map = {}
#     for line in f:
#       w_line = line.split()
#       curr_word = w_line[0]
#       word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

#   return word_to_vec_map

In [13]:
# word_to_vec_map = read_glove_vector('./glove.6B.50d.txt')
# maxLen = 150

# vocab_len = len(words_to_index)
# embed_vector_len = word_to_vec_map['moon'].shape[0]

# emb_matrix = np.zeros((vocab_len, embed_vector_len))

# for word, index in words_to_index.items():
#     embedding_vector = word_to_vec_map.get(word)
#     if embedding_vector is not None:
#         emb_matrix[index, :] = embedding_vector

# embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [14]:
def duration_prediction_model(input_shape):

  X_indices = Input(input_shape)
  # embeddings = embedding_layer(X_indices)

  # X = LSTM(128, return_sequences=True)(embeddings)

  # # X = Dropout(0.6)(X)

  # # X = LSTM(128, return_sequences=True)(X)

  # # X = Dropout(0.6)(X)

  # # X = LSTM(128)(X)

  # X = Dense(1, activation='softmax')(X)
  # print("X: ")
  # print( X)

  # model = Model(inputs=X_indices, outputs=X)

  model = Sequential()
  model.add(InputLayer(input_shape))
  model.add(LSTM(64))
  model.add(Dense(8, 'relu'))
  model.add(Dense(1, 'linear'))
  return model

In [15]:
# def conv1d_model(input_shape):

#   X_indices = Input(input_shape)

#   embeddings = embedding_layer(X_indices)

#   X = Conv1D(512,3,activation='softmax')(embeddings)
  
#   X = MaxPooling1D(3)(X)

#   X = Conv1D(256,3,activation='softmax')(X)
  
#   X = MaxPooling1D(3)(X)

#   X = Conv1D(256,3,activation='softmax')(X)
#   X = Dropout(0.8)(X)
#   X = MaxPooling1D(3)(X)

#   X = GlobalMaxPooling1D()(X)

#   X = Dense(256, activation='softmax')(X)
#   X = Dense(10, activation='sigmoid')(X)

#   model = Model(inputs=X_indices, outputs=X)

#   return model

In [16]:
model = duration_prediction_model((4, 1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17,425
Trainable params: 17,425
Non-trainable params: 0
_________________________________________________________________


In [17]:
# model_1d = conv1d_model((maxLen,))
# model_1d.summary()

In [18]:
# X_train_indices = tokenizer.texts_to_sequences(X_train)
# X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
# X_train_indices.shape
# adam = tf.keras.optimizers.Adam(learning_rate = 0.0100)
# model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
# model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

In [20]:

from tensorflow.keras.optimizers import Adam
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
X_val1, y_val1 = X_test[28:30], Y_test[28:30]
# model.fit(X_train, Y_train, validation_data=(X_val1, y_val1), epochs=100, callbacks=[cp1])
# model.save('./model')

In [ ]:
# X_test_indices = tokenizer.texts_to_sequences(X_test)
# X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
from keras.models import load_model
model = load_model('model/')
model.fit(X_train, Y_train, validation_data=(X_val1, y_val1), epochs=400, callbacks=[cp1])
model.save('./model')
# model.evaluate(X_test, Y_test)
# model_1d.evaluate(X_test_indices, Y_test)

In [27]:
# preds = model_1d.predict(X_test_indices)
# print(f"X_test = \n{X_test}\ntype = {type(X_test)}")
# print(f"X_test = \n{X_test}\ntype = {X_test.shape}")
preds = model.predict(X_test)
# print(f'Predicted duration : {preds[0]}, correct duration : {Y_test[6]}')
print(type(X_test.iloc[0]))
print(X_test.iloc[0].shape)

for i in range(len(X_test)):
    print(f'Predicted duration : {preds[i]}, correct duration : {Y_test[i]}')

<class 'pandas.core.series.Series'>
(4,)
Predicted duration : [2.0143917], correct duration : 2
Predicted duration : [3.4719455], correct duration : 4
Predicted duration : [2.9709494], correct duration : 3
Predicted duration : [2.9715176], correct duration : 3
Predicted duration : [3.898427], correct duration : 5
Predicted duration : [4.0262475], correct duration : 4
Predicted duration : [1.2427027], correct duration : 1
Predicted duration : [2.0177948], correct duration : 2


In [ ]:
# print(f'predicted duration : {preds[n]}')
# print(f'correct duration : {Y_test[n]}')

In [ ]:
preds[n]

array([0.58026505], dtype=float32)

In [ ]:
Y_test[n]

4

In [ ]:
model_1d.save_weights('./imdb_weights_con1vd.hdf5')

In [ ]:
def add_score_predictions(data):

  data['duration score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['duration score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted duration'] = 0

  data['predicted duration'] = pred_sentiment

  return data

In [ ]:
# data = add_score_predictions(data, reviews_list_idx)

In [ ]:
data

,Skill,Complexity,task_name,subtask_name,Duration,task_name without stopwords,subtask_name without stopwords,clean_task_name,clean_subtask_name
0,3,1,home page,api to validate and fetch hot products,3,home page,api validate fetch hot products,home page,api validate fetch hot products
1,2,0,home page,api to validate and fetch new products,2,home page,api validate fetch new products,home page,api validate fetch new products
2,3,1,categories page,api to validate and fetch all categories along...,3,categories page,api validate fetch categories along products b...,categories page,api validate fetch categories along products b...
3,2,0,categories page,api to validate and fetch a single category,2,categories page,api validate fetch single category,categories page,api validate fetch single category
4,2,0,categories page,api to filter and sort results,2,categories page,api filter sort results,categories page,api filter sort results
5,2,0,plp,api to validate and fetch list of products,2,plp,api validate fetch list products,plp,api validate fetch list products
6,3,1,plp,api to filter and sort results,3,plp,api filter sort results,plp,api filter sort results
7,3,1,plp,api to persist the chosen item in db for cart,3,plp,api persist chosen item db cart,plp,api persist chosen item db cart
8,3,1,plp,api to persist the chosen item in db for wishlist,3,plp,api persist chosen item db wishlist,plp,api persist chosen item db wishlist
9,1,0,pdp,api to validate and fetch product details,1,pdp,api validate fetch product details,pdp,api validate fetch product details
